In [1]:
# import TensorFlow as TF
import pandas as pd
import math
import matplotlib.pylab as plt
import numpy as np

from sklearn import datasets
from sklearn.impute import SimpleImputer # used for handling missing data


In [2]:

df = pd.read_csv('data.csv')
data_top = df.head()
copy = df.copy() 

# 1 - Remove rows where the target column (Scope1) is NaN.

#copy = copy[copy['EmissionDataPoints.CO2EquivalentsEmissionDirectScope1'].notna()] # drop all rows with NaN in target column

#copy = copy.reset_index() 



# 2 - Replace all NaN from the attribute columns and replace for mean of that column

# copy = copy.fillna(copy.mean())
# col_names = copy.columns
# sub2[col_names].fillna((sub2[col_names].mean()), inplace=True)


def create_dict(target_var, df): 
    org_ids = list(df[target_var]) 
    list_set = set(org_ids) # get unique elements
    unique = list(list_set) # place them in a list 

    df = df.groupby([target_var]) 

    dictionary = {} 

    for item in unique:
        df_id = df.get_group(item)
        dictionary[item] = df_id
    
    return dictionary

dict_id = create_dict('organization_id', copy)
print(dict_id[14346])

#print(copy.organzation_id)

#key_list = list(GTAP_sector_dict.keys())

#print(key_list)

#print(GTAP_sector_dict)

# in the dictionary, we want to replace all the NaN of the values for the means of those columns.


# # check missing values >> columns with lots of missing values are not useful. 
# missing_values_df = copy.isnull().sum().sort_values(ascending=False).head()
# print(missing_values_df)

# # replace NaN in column for mean of column with Imputer 
# imp = SimpleImputer(missing_values='NaN', strategy='mean')
# imp = imp.fit(copy)



#print(replace_NaN(GTAP_sector_dict))    



       organization_id StatementDetails.FinancialPeriodEndDate  \
29557            14346                2019-12-31T00:00:00.000Z   
66052            14346                2018-12-31T00:00:00.000Z   

       StatementDetails.FinancialPeriodFiscalYear  \
29557                                        2019   
66052                                        2018   

       StatementDetails.FinancialPeriodIsIncomplete  \
29557                                         False   
66052                                         False   

      StatementDetails.EsgOrAtdIndicator  EmissionDataPoints.AccidentalSpills  \
29557                                ESG                                  NaN   
66052                                ESG                                  NaN   

      EmissionDataPoints.AnalyticCO2EstimationMethod  \
29557                                       Reported   
66052                                         Median   

       EmissionDataPoints.AnalyticEstimatesCO2EquivalentsEmiss

In [3]:
# function for filling NaN's
def fill_nan(df, column_name):
    mean_value=df[column_name].mean()
    df[column_name].fillna(value=mean_value, inplace=True)
    return df

# function for determining percentage NaN
def ratio_nan(df, column_name):
    ratio = df[column_name].isnull().sum() / len(df)
    return ratio

# function to loop over org_ids and creat df with ratio's nan in column
def dict_ratios(dictionary, column_name):
    dict_ratios = {}
    for key in dictionary:
        ratio_key = ratio_nan(dictionary[key], column_name)
        dict_ratios[key] = ratio_key
    return dict_ratios

#print(np.array(list(dict_ratios(dict_id, 'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1').values())).mean())
#print(dict_ratios(dict_id, 'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1'))

def df_clean(df, column_name):
    for i in range(len(df[column_name])):
        if math.isnan(df[column_name][i]):
            if math.isnan(df[column_name][i+1]):
                if math.isnan(df[column_name][i+2]):
                    df.drop([i, i+1, i+2])
                else:
                    if i == 0:
                        df[column_name][0] = df[column_name][i+2]
                    else:
                        mean = (df[column_name][i-1]+df[column_name][i+2])/2
                        df[column_name][i] = mean
                    df[column_name][i+1] = mean
            else:
                if i == 0:
                    df[column_name][0] = df[column_name][i+1]
                else:
                    mean = (df[column_name][i-1] + df[column_name][i+1])/2
                    df[column_name][i] = mean
    return df

def dict_clean_mean(dictionary, column_name, ratio):
    dict_clean = {}
    for key in dictionary:
        ratio_key = ratio_nan(dictionary[key], column_name)
        if ratio_key <= ratio:
            dict_clean[key] = fill_nan(dictionary[key], column_name)
        else:
            df_key = dictionary[key]
            df_key = df_key[df_key[column_name].notna()]
            dict_clean[key] = df_key
    return dict_clean

def dict_clean_better(dictionary, column_name, ratio):
    dict_clean = {}
    for key in dictionary:
        ratio_key = ratio_nan(dictionary[key], column_name)
        if ratio_key <= ratio:
            dict_clean[key] = fill_nan(dictionary[key], column_name)
        else:
            df_key = dictionary[key]
            df_key_clean = df_clean(df_key, column_name)
            dict_clean[key] = df_key_clean
    return dict_clean

def dict_clean_worst(dictionary, column_name):
    for key in dictionary:
        for i in range(len(dictionary[key][column_name])):
            if math.isnan(dictionary[key][column_name][i]):
                if math.isnan(dictionary[key][column_name][i+1]):
                    if math.isnan(dictionary[key][column_name][i+2]):
                        dictionary[key].drop([i, i+1, i+2])
                    else:
                        if i == 0:
                            dictionary[key][column_name][0] = dictionary[key][column_name][i+2]
                        else:
                            mean = (dictionary[key][column_name][i-1]+dictionary[key][column_name][i+2])/2
                            dictionary[key][column_name][i] = mean
                            dictionary[key][column_name][i+1] = mean
                else:
                    if i == 0:
                        dictionary[key][column_name][0] = dictionary[key][column_name][i+1]
                    else:
                        mean = (dictionary[key][column_name][i-1] + dictionary[key][column_name][i+1])/2
                        dictionary[key][column_name][i] = mean
    return dict_clean


        

# print(ratio_nan(dict_id[6230], 'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1'))
dict_nan = dict_clean_better(dict_id, 'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1', 0.6)

df_6230 = dict_id[6230]
df_clean_6230 = dict_nan[6230]

df_72 = dict_id[72]
df_clean_72 = dict_nan[72]

#print(df_72['EmissionDataPoints.CO2EquivalentsEmissionDirectScope1'])
#print(df_clean_72['EmissionDataPoints.CO2EquivalentsEmissionDirectScope1'])




/Users/julie/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


20060    42051.0
25387    39982.0
57574    41016.5
Name: EmissionDataPoints.CO2EquivalentsEmissionDirectScope1, dtype: float64
20060    42051.0
25387    39982.0
57574    41016.5
Name: EmissionDataPoints.CO2EquivalentsEmissionDirectScope1, dtype: float64


In [12]:
#copy.groupby("organization_id")['EmissionDataPoints.CO2EquivalentsEmissionDirectScope1'].plot(x='StatementDetails.FinancialPeriodFiscalYear',y='EmissionDataPoints.CO2EquivalentsEmissionDirectScope1')
# .mean()

#print(list(copy.columns))

def find_correlation(df, target_value, column_name):
    # way to determine if columns have correlation
    df_target = df[target_value]
    df_corr = df[column_name]
    corr = df.corrwith(df_target)
    return corr

print(find_correlation(dict_id[6230], 'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1', 'EmissionDataPoints.NonHazardousWaste'))



organization_id                                                         NaN
StatementDetails.FinancialPeriodFiscalYear                         0.899983
StatementDetails.FinancialPeriodIsIncomplete                            NaN
EmissionDataPoints.AccidentalSpills                                     NaN
EmissionDataPoints.AnalyticEstimatesCO2EquivalentsEmissionTotal    0.833735
                                                                     ...   
WorkforceIndicators.NetEmploymentCreation                          0.493391
WorkforceIndicators.SalaryGap                                           NaN
WorkforceIndicators.TrainingAndDevelopmentPolicy                        NaN
WorkforceIndicators.TrainingCostsPerEmployee                            NaN
WorkforceIndicators.WagesWorkingConditionControversies                  NaN
Length: 179, dtype: float64


/Users/julie/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/Users/julie/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
